---
title: Sketch and Precondition
description: High-accuracy linear regression using randomized sketching for preconditioning iterative methods
keywords: [sketch and precondition, preconditioning, iterative methods, LSQR, condition number, sketched QR, convergence acceleration]
numbering:
  equation:
    enumerator: 4.%s
    continue: true
  proof:theorem:
    enumerator: 4.%s
    continue: true
  proof:algorithm:
    enumerator: 4.%s
    continue: true
  proof:definition:
    enumerator: 4.%s
    continue: true
  proof:proposition:
    enumerator: 4.%s
    continue: true
---

The sketch-and-precondition approach provides a powerful alternative to both direct factorization methods and the [sketch-and-solve](sketch-and-solve.ipynb) paradigm for solving {eq}`task-regression`.
Unlike sketch-and-solve, which may sacrifice accuracy for speed, sketch-and-precondition maintains high accuracy while achieving significant computational speedups.

The key insight behind sketch-and-precondition is to use a randomized sketch to construct a preconditioner that improves the conditioning the {eq}`task-regression`.

## Preconditioned Iterative Methods

Given any invertivle matrix $\vec{M}\in\R^{d\times d}$ (called a *preconditioner*), solving {eq}`task-regression` is equivalent to solving
```{math}
:label: eqn-preconditioned-regression
\min_{\vec{y}} \|\vec{A}\vec{M}\vec{y} - \vec{b}\|
,\quad
\vec{x} = \vec{M}\vec{y}.
```
The convergence of iterative methods applied to the preconditioned system depends on the condition number of the preconditioned matrix $\vec{A}\vec{M}$.

:::{prf:theorem}

Suppose LSQR is applied to {ref}`eqn-preconditioned-regression` for $t$ iterations to produce an approximate solution $\widehat{\vec{x}}_t$.
Then, for some 
\begin{equation*}
t = O \left( \cond(\vec{A}\vec{M}) \log \left( \frac{1}{\varepsilon} \right) \right),
\end{equation*}
it holds that
\begin{equation*}
\| \vec{A}(\vec{x}^* - \widehat{\vec{x}}_t) \| \leq \varepsilon \|\vec{b} - \vec{A}\vec{x}^*\|.
\end{equation*}
This requires $t$ matrix-vector products with $\vec{A}$, $\vec{A}^\T$, $\vec{M}$ and $\vec{M}^\T$, in addition to $O(tn)$ flops.
:::



## Randomized Preconditioning

The basic sketch-and-precondition algorithm follows a two-stage approach: first construct a preconditioner using a randomized sketch, then solve the preconditioned system using an iterative method.

:::{prf:algorithm} Sketch and Precondition
:label: sketch-and-precondition

**Input:** $\vec{A}\in\R^{n\times d}$, $\vec{b}\in\R^n$, sketching dimension $k$, tolerance $\varepsilon$

1. Get $\vec{Q}_1,\vec{R}_1 = \Call{Sketched-QR}(\vec{A},k)$
2. Form preconditioner $\vec{M} = \vec{R}^{-1}$
3. Set $\widehat{\vec{x}} = \Call{Iterative-Method}(\vec{A},\vec{b},\vec{M})$

**Output:** $\vec{x}$
:::

Any version of [Sketched-QR algorithm](../03-QR-Factorization/randomized-cholesky-qr.ipynb) that produces subspace embedding can be used.

:::{prf:theorem}

Suppose $\vec{A}$ is full-rank and the sketch $\vec{S}$ used by the [Sketched-QR algorithm](../03-QR-Factorization/randomized-cholesky-qr.ipynb) is an $\varepsilon$-subspace embedding for $\vec{A}$.
Then 
\begin{equation*}
\cond(\vec{A}\vec{M}) \leq  \frac{1+\varepsilon}{1-\varepsilon}.
\end{equation*}
:::

:::{prf:proof}
:class: dropdown
:enumerated: false

Recall $\vec{A} = \vec{Q}\vec{R}$.
By {eq}`sketched-qr-well-conditioned`,
\begin{equation*}
\cond(\vec{A}\vec{M}) = \cond(\vec{A}\vec{R}^{-1}) = \cond(\vec{Q}) \leq (1+\varepsilon)/(1-\varepsilon)
\end{equation*}
:::